In [1]:
import importlib
import math

import numpy as np
import pandas as pd

import preprocessing
importlib.reload(preprocessing);

In [2]:
filename = './data/axaf_mrh_ddea_freq_tot.csv' 
    
features = [
    'POL_mtcapass',
    'EXT_superficie_m2_num',
    'EXT_dens_pop',
    'EXT_part_respr_1p',
    'EXT_part_respr_5p',
    'EXT_part_secocc',
    'EXT_part_surf_inf40',
    'EXT_part_surf_plus100',
    'EXT_part_rev_15K29k',
    'EXT_part_men_lochlm',
    'EXT_part_men_pror',
    'EXT_part_pop_18a24a',
    'EXT_part_pop_65a79a',
    'EXT_part_respr_chauf_aut',
    'EXT_part_respr_chauf_col',
    'EXT_part_respr_chauf_ind',
    'EXT_part_respr_chauf_indelec',
    'EXT_part_repsr_sdb1',
    'EXT_part_repsr_sdb2',
    'EXT_part_lgt_co',
    'EXT_part_lgt_vac',
    'EXT_part_pop_15dipsup',
    'EXT_part_pop_15ssdip',
    'EXT_gdeville',
    'AUTV_prix_sra_max',
    'AUTV_prix_sra_min',
    'AUTV_rm',
    'AUTV_segment_max',
    'AUTV_segment_min',
    'CLI_banque',
    'POL_cdopmrh',
    'POL_fract',
    'POL_cdregion',
    'HAB_cdresid',
    'CLI_sex',
    'POL_distrib',
    'AUTV_energie_es',
    'AUTV_energie_go',
    'AUTV_k8000_n',
    'AUTV_k8000_o',
    'AUTV_prix_sra_ad',
    'AUTV_prix_sra_ef',
    'AUTV_prix_sra_gi',
    'AUTV_prix_sra_jl',
    'AUTV_prix_sra_mn',
    'AUTV_prix_sra_supo',
    'AUTV_segment_0',
    'AUTV_segment_a',
    'AUTV_segment_b',
    'AUTV_segment_h',
    'AUTV_segment_m1',
    'AUTV_segment_m2',
    'AUTV_nb_rc_12',
    'AUTV_nb_rc_34',
    'AUTV_nb_rc_56',
    'AUTV_nb_rc_78',
    'AUTV_nb_rc_sup9',
    'AUTV_nb_vol_ab',
    'AUTV_nb_vol_cd',
    'AUTV_nb_vol_ef',
    'AUTV_nb_vol_gh',
    'AUTV_nb_vol_ij',
    'AUTV_nb_vol_supk',
    'HAB_inser',
    'CLI_indice',
    'AUTA_tot1a',
    'AUTA_tot2a',
    'AUTA_tot3a',
    'AUTA_tot4a',
    'POL_tx_objv',
    'POL_tr_capass',
    'CLI_age',
    'annee',
    'CLI_nb_contrat',
    'CLI_nb_contrat_atr',
    'CLI_nb_contrat_aup',
    'CLI_nb_contrat_bqe',
    'CLI_nb_contrat_epa',
    'CLI_nb_contrat_hab',
    'CLI_nb_contrat_mot',
    'CLI_nb_contrat_mrp',
    'CLI_nb_contrat_pf',
    'CLI_nb_contrat_pju',
    'CLI_nb_contrat_prv',
    'CLI_nb_contrat_san',
    'HAB_nb_pieces',
    'CLI_nb_enfant',
    'POL_fr',
    'AUTA_b50_n',
    'AUTA_b50_o',
    'AUTA_rm_50',
    'AUTA_rm_5060',
    'AUTA_rm_6070',
    'AUTA_rm_7090',
    'AUTA_rm_inf50',
    'AUTA_rm_sup90',
    'HAB_surf_dep',
    'HAB_inhab',
    'AUTV_nb_rc',
    'AUTV_nb_vol',
    'POL_pac',
    'POL_paj',
    'POL_pan',
    'POL_pap',
    'POL_par',
    'CLI_prod_entree',
    'HAB_qual',
    'POL_ran',
    'CLI_top_atr',
    'CLI_top_aup',
    'CLI_top_bqe',
    'CLI_top_epa',
    'CLI_top_hab',
    'CLI_top_mot',
    'CLI_top_mrp',
    'CLI_top_pf',
    'CLI_top_pju',
    'CLI_top_prv',
    'CLI_top_san',
    'HAB_top_jardin',
    'CLI_tr_age',
    'CLI_tr_ancli',
    'CLI_tr_mtvi',
    'CLI_tr_pm',
    'HAB_habit_X_qual_X_inhab',
    'POL_mtobv',
    'POL_tr_tx_objv',
    'CLI_cspf',
    'REG_zoneAxa_a',
    'REG_zoneAxa_m',
    'Random',
    'HAB_anclg',
    'CLI_sit_fam2',
    'AUTA_rm',
    'HAB_hab',
    'HAB_habit',
    'POL_delec',
    'POL_vol',
    'CLI_tr_age_X_CSPf',
    'ddea_quant_freq_20',
    'ddea_quant_cm_10',
    'ddem_quant_freq_20',
    'ddem_quant_cm_20',
    'clim_quant_pp_20',
    'vol_quant_freq_20',
    'vol_quant_cm_20',
    'bdg_quant_freq_20',
    'bdg_quant_cm_20',
    'catnat_quant_pp_10',
    'fire_quant_freq_20',
    'HAB_habit_nbpieces',
    'HAB_qual_inhab',
    'POL_distrib_code',
    'Zone_inc_cm_iris',
    'Zone_inc_freq_iris',
    'zone_vol_cm_insee',
    'zone_vol_freq_insee',
]

exposure = 'anpol'

targets = [
    # costs
    'chargeBDGm',
    'chargeCATNATm',
    'chargeCLIMm',
    'chargeDDEm',
    'chargeDELECm',
    'chargeINCattri',
    'chargeINCgr',
    'chargeRCm',
    'chargeVOLm',

    # count
    'nbsinBDG',
    'nbsinCATNAT',
    'nbsinCATNAT2',
    'nbsinCLIM',
    'nbsinDDE',
    'nbsinDELEC',
    'nbsinINC',
    'nbsinINCattri',
    'nbsinINCgr',
    'nbsinRC',
    'nbsinVOL',
]

In [3]:
def data_transform(field, value):
    if field == 'POL_mtcapass':
        return int(math.log(float(value)))
    elif field == 'EXT_superficie_m2_num':
        return int(math.log(float(value)+2))
    return value

In [4]:
metadata = preprocessing.Metadata("data", "mrh")
metadata.set_features(features)
metadata.set_targets(targets)
metadata.set_exposure(exposure)

In [ ]:
metadata.process(filename, data_transform)

Starting data importation.
Importing 4 459 542 lines.


In [ ]:
metadata = preprocessing.Metadata("data", "mrh")
metadata.load()

In [6]:
observations = np.memmap('./data/observations.dat', np.dtype('u1'))
exposure_data = np.memmap('./data/exposure.dat', np.dtype('float32'))
targets_data = np.memmap('./data/targets.dat', np.dtype('float32'))

In [5]:
metadata.count_features()

157

In [6]:
metadata.count_modalities()

1471

In [7]:
metadata.count_modalities_per_feature()

[7,
 12,
 21,
 12,
 21,
 13,
 17,
 21,
 21,
 14,
 21,
 12,
 16,
 17,
 15,
 21,
 21,
 9,
 9,
 21,
 14,
 18,
 20,
 11,
 7,
 7,
 2,
 7,
 7,
 23,
 7,
 3,
 6,
 2,
 3,
 8,
 3,
 4,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 6,
 7,
 11,
 11,
 12,
 6,
 19,
 94,
 5,
 45,
 18,
 9,
 11,
 11,
 38,
 6,
 7,
 3,
 4,
 5,
 5,
 10,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 1,
 2,
 3,
 2,
 2,
 7,
 2,
 2,
 2,
 2,
 2,
 10,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 9,
 9,
 10,
 10,
 9,
 20,
 6,
 12,
 12,
 17,
 10,
 4,
 4,
 6,
 1,
 2,
 2,
 2,
 73,
 21,
 11,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 11,
 21,
 18,
 4,
 4,
 8,
 21,
 16,
 21]

In [8]:
metadata.features

['POL_mtcapass',
 'EXT_superficie_m2_num',
 'EXT_dens_pop',
 'EXT_part_respr_1p',
 'EXT_part_respr_5p',
 'EXT_part_secocc',
 'EXT_part_surf_inf40',
 'EXT_part_surf_plus100',
 'EXT_part_rev_15K29k',
 'EXT_part_men_lochlm',
 'EXT_part_men_pror',
 'EXT_part_pop_18a24a',
 'EXT_part_pop_65a79a',
 'EXT_part_respr_chauf_aut',
 'EXT_part_respr_chauf_col',
 'EXT_part_respr_chauf_ind',
 'EXT_part_respr_chauf_indelec',
 'EXT_part_repsr_sdb1',
 'EXT_part_repsr_sdb2',
 'EXT_part_lgt_co',
 'EXT_part_lgt_vac',
 'EXT_part_pop_15dipsup',
 'EXT_part_pop_15ssdip',
 'EXT_gdeville',
 'AUTV_prix_sra_max',
 'AUTV_prix_sra_min',
 'AUTV_rm',
 'AUTV_segment_max',
 'AUTV_segment_min',
 'CLI_banque',
 'POL_cdopmrh',
 'POL_fract',
 'POL_cdregion',
 'HAB_cdresid',
 'CLI_sex',
 'POL_distrib',
 'AUTV_energie_es',
 'AUTV_energie_go',
 'AUTV_k8000_n',
 'AUTV_k8000_o',
 'AUTV_prix_sra_ad',
 'AUTV_prix_sra_ef',
 'AUTV_prix_sra_gi',
 'AUTV_prix_sra_jl',
 'AUTV_prix_sra_mn',
 'AUTV_prix_sra_supo',
 'AUTV_segment_0',
 'AUTV

In [9]:
metadata.modalities

{'POL_mtcapass': [9, 7, 8, 11, 10, 6, 12],
 'EXT_superficie_m2_num': [0, 11, 17, 14, 12, 15, 13, 10, 16, 19, 18, 9],
 'EXT_dens_pop': ['-1',
  '26253.76',
  '110.89',
  '493.87',
  '14095.55',
  '0',
  '3693.76',
  '895.59',
  '6855.8',
  '9468.64',
  '310.58',
  '212.1',
  '1554.96',
  '32.07',
  '5084.28',
  '2572.86',
  '45.79',
  '63.06',
  '83.5',
  '151.62',
  '19.28'],
 'EXT_part_respr_1p': ['-1',
  '0.16',
  '0.03',
  '0.02',
  '0',
  '0.09',
  '0.23',
  '0.04',
  '0.01',
  '0.07',
  '0.12',
  '0.05'],
 'EXT_part_respr_5p': ['-1',
  '0',
  '0.46',
  '0.16',
  '0.24',
  '0.37',
  '0.1',
  '0.54',
  '0.13',
  '0.2',
  '0.07',
  '0.34',
  '0.31',
  '0.4',
  '0.28',
  '0.65',
  '0.48',
  '0.51',
  '0.6',
  '0.43',
  '0.57'],
 'EXT_part_secocc': ['-1',
  '0.05',
  '0.04',
  '0.01',
  '0',
  '0.02',
  '0.03',
  '0.51',
  '0.06',
  '0.09',
  '0.17',
  '0.12',
  '0.27'],
 'EXT_part_surf_inf40': ['-1',
  '0.38',
  '0.08',
  '0.06',
  '0.21',
  '0.12',
  '0.17',
  '0.28',
  '0.04',
  '0.

In [10]:
metadata.fields

['EXT_dens_pop',
 'EXT_part_respr_1p',
 'EXT_part_respr_5p',
 'EXT_part_secocc',
 'EXT_part_surf_inf40',
 'EXT_part_surf_plus100',
 'EXT_part_rev_15K29k',
 'EXT_part_men_lochlm',
 'EXT_part_men_pror',
 'EXT_part_pop_18a24a',
 'EXT_part_pop_65a79a',
 'EXT_part_respr_chauf_aut',
 'EXT_part_respr_chauf_col',
 'EXT_part_respr_chauf_ind',
 'EXT_part_respr_chauf_indelec',
 'EXT_part_repsr_sdb1',
 'EXT_part_repsr_sdb2',
 'EXT_part_lgt_co',
 'EXT_part_lgt_vac',
 'EXT_part_pop_15dipsup',
 'EXT_part_pop_15ssdip',
 'EXT_gdeville',
 'AUTV_prix_sra_max',
 'AUTV_prix_sra_min',
 'AUTV_rm',
 'AUTV_segment_max',
 'AUTV_segment_min',
 'CLI_banque',
 'POL_cdopmrh',
 'POL_fract',
 'POL_cdregion',
 'HAB_cdresid',
 'CLI_sex',
 'POL_distrib',
 'AUTV_energie_es',
 'AUTV_energie_go',
 'AUTV_k8000_n',
 'AUTV_k8000_o',
 'AUTV_prix_sra_ad',
 'AUTV_prix_sra_ef',
 'AUTV_prix_sra_gi',
 'AUTV_prix_sra_jl',
 'AUTV_prix_sra_mn',
 'AUTV_prix_sra_supo',
 'AUTV_segment_0',
 'AUTV_segment_a',
 'AUTV_segment_b',
 'AUTV_segm

In [11]:
metadata.get_offsets()

array([   0,    7,   19,   40,   52,   73,   86,  103,  124,  145,  159,
        180,  192,  208,  225,  240,  261,  282,  291,  300,  321,  335,
        353,  373,  384,  391,  398,  400,  407,  414,  437,  444,  447,
        453,  455,  458,  466,  469,  473,  475,  477,  479,  481,  483,
        485,  487,  489,  491,  493,  495,  497,  499,  501,  503,  506,
        508,  510,  512,  514,  516,  518,  520,  522,  524,  526,  532,
        539,  550,  561,  573,  579,  598,  692,  697,  742,  760,  769,
        780,  791,  829,  835,  842,  845,  849,  854,  859,  869,  871,
        873,  875,  877,  879,  881,  883,  885,  886,  888,  891,  893,
        895,  902,  904,  906,  908,  910,  912,  922,  924,  926,  928,
        930,  932,  934,  936,  938,  940,  942,  944,  946,  948,  950,
        959,  968,  978,  988,  997, 1017, 1023, 1035, 1047, 1064, 1074,
       1078, 1082, 1088, 1089, 1091, 1093, 1095, 1168, 1189, 1200, 1221,
       1242, 1263, 1284, 1305, 1326, 1347, 1358, 13

In [12]:
metadata.get_feature_index('HAB_inhab')

97

In [20]:
list(metadata.get_feature_range('HAB_inhab'))

[891, 892]

In [21]:
metadata.exposure

'anpol'

In [22]:
metadata.get_unused_fields()

['chargeCATNATm',
 'chargeDDEm',
 'chargeBDGm',
 'nbsinCLIM',
 'top_sin_extreme',
 'chargeCLIMm',
 'anpolVOL',
 'nbsinCATNAT',
 'nbsinDELEC',
 'newid',
 'nbsinINC',
 'anpolDEL',
 'nbsinVOL',
 'nbsinCATNAT2',
 'chargeINCattri',
 'nbsinINCattri',
 'top_forfaitRC',
 'top_forfaitDELEC',
 'chargeDELECm',
 'nbsinRC',
 'nbsinBDG',
 'top_forfaitDDE',
 'chargeVOLm',
 'top_forfaitINC',
 'top_forfaitCLIM',
 'chargeINCgr',
 'top_forfaitBDG',
 'top_clim10M',
 'chargeRCm',
 'top_forfaitVOL',
 'nbsinINCgr']

In [19]:
(np.log(pd.read_csv(filename, nrows=100000)['POL_mtcapass'])).astype('int').unique() 

array([ 9,  7,  8, 11, 10,  6, 12, 13])

In [18]:
df = pd.read_csv(filename, nrows=100000)
for i in list(df):
    if len(df[i].unique()) > 200:
        print("'" + i + "',")

'POL_mtcapass',
'anpol',
'chargeDDEm',
'chargeVOLm',
'chargeRCm',
'chargeBDGm',
'anpolDEL',
'anpolVOL',
'EXT_superficie_m2_num',
'newid',


In [19]:
df.EXT_superficie_m2_num.describe()

count    1.000000e+05
mean     2.648645e+06
std      9.980882e+06
min     -1.000000e+00
25%      1.098178e+05
50%      3.060705e+05
75%      8.426892e+05
max      3.678477e+08
Name: EXT_superficie_m2_num, dtype: float64